# DST Airlines

[LINK](https://docs.google.com/document/d/1wyo5QOe8UZZE0_fm0PNdc_bTUNqjyrBMHSHwH-JfSEM/edit)

Description détaillée :  
De nos jours, il est possible d’avoir des informations sur les vols dans le monde entier et de traquer en temps réel un avion. Nous pouvons observer ce site en guise d’exemple.  
Le but ici est de s’en approcher le plus possible en passant par des API de différentes compagnies aériennes.

Etape | Description | Modules / Masterclass / Templates | Objectif | Conditions de validation du projet
----- | ----------- | --------------------------------- | -------- | ---------------------------------
1 | Exploration de données non structurées | Passer par l’API de Lufthansa pour récupérer des données sur les vols | 103 - Introduction aux APIs de données | Un fichier explicatif du traitement et des différentes données accessibles (doc / pdf). Un exemple de données collectées.


[Lufthansa Open API Documentation](https://developer.lufthansa.com/docs)

[LH OpenAPI testing](https://developer.lufthansa.com/io-docs)

[Public API Method Details](https://developer.lufthansa.com/docs/read/api_details)

!!! [Un repo GIT englobe déjà les requêtes à l'API, il suffirait de le forker](https://github.com/arossmann/lh_open_api)

In [1]:
# imports

import sys
sys.path.append('config')

import lh_access as API # API credentials, not on git
import request




Getting countries

In [2]:
# req = request.get('https://api.lufthansa.com/v1/flight-schedules/flightschedules/cargo?airlines=LH&startDate=01NOV23&endDate=02NOV23&daysOfOperation=1234567&timeMode=UTC&origin=BER')
# print(req)
req = request.get('https://api.lufthansa.com/v1/mds-references/airports/FRA')
with open('files/countries.json', 'w') as countries:
    countries.write(req)



Getting cities

In [11]:
req = request.get('https://api.lufthansa.com/v1/mds-references/cities')
with open('files/cities.json', 'w') as cities:
    cities.write(req)

TypeError: write() argument must be str, not None

Getting airports

In [10]:
req = request.get('https://api.lufthansa.com/v1/mds-references/airports?LHoperated=1')
with open('files/aiports.json', 'w') as airports:
    airports.write(req)